## Load PyEpiDAG

In [1]:
import epidag as dag
import numpy as np

## Define a DAG

Compose a script

```
PCore Exp1 {
    # Definitions of nodes
}
```

Then, .. parse to a DAG

In [2]:
script = '''
PCore Exp1 {
    n
    a = 0.5 
    b ~ beta(1, 1)
    c = min(a, b)
    y ~ binom(n, c)
    z = f(a,b)
}
'''

js = dag.bn_script_to_json(script)
bn = dag.BayesianNetwork(js)
bn

Name:	Exp1
Nodes:
	n
	b ~ beta(1,1)
	a = 0.5
	z = f(a, b)
	c = min(a,b)
	y ~ binom(n,c)

### Single value variable


> VariableName = Number 


In [3]:
SingleValue = bn['a']
print('Node \'a\'')
print('Definition:', SingleValue)

print('\nFind the value')
print(SingleValue())

Node 'a'
Definition: a = 0.5

Find the value
0.5


### Exogenous variable

> VariableName

In [4]:
Exogenous = bn['n']
print('Node \'n\'')
print('Definition:', Exogenous)

print('\nFind the value; must append external resources')
print(Exogenous({'n': 5}))

Node 'n'
Definition: n

Find the value; must append external resources
5


### Random variable

> VariableName ~ p(...)

** p(...) **: a probabilidy density/mass function

In [5]:
Random = bn['b']
print('Node \'b\'')
print('Definition:', Random)

print('\nSample a value')
print(Random())

Node 'b'
Definition: b ~ beta(1,1)

Sample a value
0.225970230493


### Equation output

> OutputName = g(...)

** g(...) **: a mathematical function

In [6]:
Equation = bn['c']
print('Node \'c\'')
print('Definition:', Equation)

parents = {
    'a': SingleValue(), 
    'b': Random() 
}

print('\nCalculate a value')
print(Equation(parents))

Node 'c'
Definition: c = min(a,b)

Calculate a value
0.451370679881


### Pseudo Node
Pseudo nodes are the nodes which can not be implemented in simulation but in relation inference.

> VariableName = f(...)

** f(...) **: a pseudo function start with ** f ** and a list of parent variables follows

In [7]:
Pseudo = bn['z']
print('Node \'z\'')
print('Definition:', Pseudo)

parents = {
    'a': SingleValue(), 
    'b': Random()
}
print('Parents:', Pseudo.Parents)

try:
    Pseudo(parents)
except AttributeError:
    print('Pseudo nodes do not provide any implementation')

Node 'z'
Definition: z = f(a, b)
Parents: {'a', 'b'}
Pseudo nodes do not provide any implementation


## For simulation model

The **'epidag.simulation'** is used to provide tools for simulation modelling. 

### Reasons to use 'epidag.simulation'
- The parameters of my model have complicated interactions among each other.
- My model includes many random effects, so I don't want to fix the parameters in the begining.
- I don't want to rebuild my model after the parameters changed. (Intervention analysis).
- My study include Monte Carlo inference and model fitting, so I need a convienant interface.


### SimulationCore
SimulationCore is a type of object carrying all the definition of a parameter model.


### ParameterCore
ParameterCore is a type of object can be use directly in a simulation model. A ParameterCore can be generated from its parent ParameterCore or a SimulationCore. After a ParameterCore instantiated 1) the fixed nodes are assigned, 2) the random nodes are ready to be used.

### Purposed workflow

Monte Carlo simulation
1. Prepare a SimulationCore
2. For each iteration, generate a ParameterCore
3. Build simulation models with the ParamterCores
4. Collect results and analysis (summarise or fit to data)

### Example 1. Basic syntex and function, Gaussian distribution

This example shows a normal variable ($x$) with a fixed mean value ($mu_x$) and a standard deviation ($sd$) distributed from an exponential distribution

#### Step 1. generate a simulation given a Bayesian network

In [8]:
script = '''
PCore Exp1 {
   mu_x = 0
   sd ~ exp(1)
   x ~ norm(mu_x, sd)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn)

#### Step 2. Instantiate a ParameterCore which 

- Hyper-parameter ($sd$) is fixed to a certain value
- A sampler of the leaf variable ($x$) is prepared

In [9]:
pc = sc.generate(nickname='exp2')
pc

sd: 1.27011, mu_x: 0

#### Step 3. Get sampler x from the ParameterCore and provide it to a simulation model.

- The sampler can be used repeatly
- You don't need to refer to its hyper-parameters while using

In [10]:
x = pc.get_sampler('x')
x

Actor x (norm(mu_x,sd)) on exp2

In [11]:
x(), np.mean([x() for _ in range(1000)])

(0.10198943534198453, 0.013835560445431527)

#### Step 4. Intervention

You can set impulse on the ParameterCore. Then, 
- The impulse will be passed down to its downstream variables
- You don't need to do anything to the sample  

In [12]:
pc.impulse({'sd': 5, 'mu_x': 100})
pc

sd: 5, mu_x: 100

In [13]:
x(), np.mean([x() for _ in range(1000)])

(92.33423572375635, 100.13785351730937)

### Example 2. Random effects, Beta-Binomial model

Example 2 is a standard example in Baysian inference. Beta-Binomial model are used to model count data ($x$) with a latent variable, probability ($prob$). 


> dag.as_simulation_core(bn, random=[...])

The option **random** defined variables which we do not want then be fixed during ParameterCore instantiation

In [25]:
script = '''
Pcore BetaBinom {
    alpha = 1
    beta = 1
    n
    prob ~ beta(alpha, beta)
    x ~ binom(n, prob)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn, random=['prob'])

Since the variable $n$ is an exogenous variable, we introduce it to new ParameterCore by **exo={...}**.

To be noticed that, $prob$ has been set as a random effect, so the variable will be requested whenever new $x$ is requested

In [29]:
pc = sc.generate('exp1', exo={'n': 100})
pc

n: 100, beta: 1, alpha: 1

Again, we get a sampler $x$ for the generated ParameterCore

In [31]:
x = sg.get_sampler('x')
x()

13

**list_all** option print all variables used to sample outcome variable. You can see $prob$ is not a fixed variable

In [34]:
x(list_all=True)

(90, {'alpha': 1, 'beta': 1, 'n': 100, 'prob': 0.93505637848964274})

In [38]:
x(list_all=True)

(26, {'alpha': 1, 'beta': 1, 'n': 100, 'prob': 0.3326761755756778})

### Example 3. Exposed variables,  Regression model

In [ ]:
script = '''
Pcore Regression {
    alpha = 1
    beta = 1
    n
    prob ~ beta(alpha, beta)
    x ~ binom(n, prob)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn, random=['prob'])

In [18]:
dag.form_hierarchy(bn)
dag.analyse_node_type(bn, dag.form_hierarchy(bn), report=True)

Group None
Must be fixed: ['beta', 'alpha', 'n']
Can be random: ['prob']
Can be actors: ['x']


{None: (['beta', 'alpha', 'n'], ['prob'], ['x'])}

### Example 3: BMI model

In [19]:

script = '''
PCore BMI {
    b0 ~ norm(12, 1)
    b1 = 0.5
    pf ~ beta(8, 20)
    foodstore ~ binom(100, pf)
    b0r ~ norm(0, .01)
    ageA ~ norm(20, 3)
    ageB ~ norm(30, 2)
    ps ~ beta(5, 6)
    sexA ~ cat({'m': ps, 'f': 1-ps})
    muA = b0 + b0r + b1*ageA
    bmiA ~ norm(muA, sd)
    sdB = sd * 0.5
    muB = b0 + b0r + b1*ageB
    bmiB ~ norm(muB, sdB)
}
'''


bn = dag.bn_from_script(script)

hie = {
    'country': ['area'],
    'area': ['b0r', 'pf', 'ps', 'foodstore', 'agA', 'agB'],
    'agA': ['bmiA', 'ageA', 'sexA'],
    'agB': ['bmiB', 'ageB']
}

sc = dag.as_simulation_core(bn, hie,
                            root='country',
                            random=['muA'],
                            out=['foodstore', 'bmiA', 'bmiB'])

pc = sc.generate('Taiwan', {'sd': 1})
pc_taipei = pc.breed('Taipei', 'area')
pc_taipei.breed('A1', 'agA')
pc_taipei.breed('A2', 'agA')
pc_taipei.breed('B1', 'agB')
pc_taipei.breed('B2', 'agB')

ageB: 28.6247, muB: 26.1396

In [20]:
pc_taipei.SG.ActorBlueprints

[ActorBlueprint(Name='foodstore', Type='f', TypeH='s', Flow=None)]

In [21]:
root = dag.form_hierarchy(bn, hie)
root.print()

country(sdB, b1, b0, sd)
-- area(b0r, ps, pf, foodstore)
---- agB(ageB, bmiB, muB)
---- agA(muA, bmiA, sexA, ageA)


In [22]:
pc.impulse({'b0r': 10})
pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 11.828)
-- Taipei (ps: 0.333148, b0r: 10, pf: 0.364774)
---- A1 (sexA: f, ageA: 23.3601)
---- A2 (sexA: f, ageA: 23.8528)
---- B1 (ageB: 28.8021, muB: 36.229)
---- B2 (ageB: 28.6247, muB: 36.1403)


In [23]:
pc.impulse(['b0r'])
pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 11.828)
-- Taipei (ps: 0.333148, b0r: 0.015668, pf: 0.364774)
---- A1 (sexA: f, ageA: 23.3601)
---- A2 (sexA: f, ageA: 23.8528)
---- B1 (ageB: 28.8021, muB: 26.2447)
---- B2 (ageB: 28.6247, muB: 26.156)


### Example 4: A simple agent-based model 

In [24]:
script = '''
PCore BMI {
    muA = 10
    bmiA ~ norm(muA, sd)
    muB ~ unif(10, 30)
    bmiB ~ norm(muB, sd)
}
'''


bn = dag.bn_from_script(script)

hie = {
    'agA': ['muA', 'bmiA'],
    'agB': ['muB', 'bmiB']
}

dag.form_hierarchy(bn, hie).print()

root(sd)
-- agB(bmiB, muB)
-- agA(bmiA, muA)


Actors hoisting


Compound: Have any non-fixed mediator in the same group
Y: Parent 
N: Child

Single: All parent nodes are fixed
Y: Parent
N: Child

Frozen: All parent ndoes are fixed
Y: Child if have parents else Parent
N: Child

